In [1]:
import sys
sys.path.append("../")
from plotting import CandlePlot
import pandas as pd

In [2]:
from technicals.patterns import apply_patterns

In [3]:
df = pd.read_pickle("../data/GBP_JPY_H1.pkl")

In [4]:
df_an = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [ ]:
df_an.tail()

In [ ]:
# direction = df_an.mid_c - df_an.mid_o
# body_size = abs(direction)
# direction = [1 if x >= 0 else -1 for x in direction]
# full_range = df_an.mid_h - df_an.mid_l
# body_perc = (body_size / full_range) * 100
# # On récupère la valeur max(upper) ou min(lower) entre le close ou le open
# body_lower = df_an[['mid_c', 'mid_o']].min(axis=1)
# body_upper = df_an[['mid_c', 'mid_o']].max(axis=1)

# body_bottom_perc = ((body_lower - df_an.mid_l) / full_range) * 100
# body_top_perc = ((df_an.mid_h - body_lower) / full_range) * 100

In [ ]:
# # On crée de nouvelles colonnes dans le dataframe pour les upper et lower
# df_an['body_lower'] = body_lower
# df_an['body_upper'] = body_upper
# df_an['body_bottom_per'] = body_bottom_perc
# df_an['body_top_perc'] = body_top_perc
# df_an['body_perc'] = body_perc

In [5]:
df_an = apply_patterns(df_an)

In [6]:
df_an.describe()

,mid_o,mid_h,mid_l,mid_c,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,direction,body_size,low_change,high_change,body_size_change,mid_point,mid_point_prev_2
count,37317.000000,37317.000000,37317.000000,37317.000000,37317.000000,37317.000000,37310.000000,37310.000000,37310.000000,37317.000000,37317.000000,37316.000000,37316.000000,3.731200e+04,37317.000000,37315.000000
mean,146.842542,146.972087,146.709094,146.843464,146.779713,146.906293,28.984499,71.015501,43.827541,0.030576,0.126580,0.000337,0.000327,inf,146.840590,146.839968
std,9.254875,9.264053,9.243728,9.254964,9.249713,9.259690,20.893942,20.893942,25.509481,0.999546,0.150904,0.128644,0.121089,NaN,9.253363,9.253221
min,125.038000,125.431000,124.044000,125.042000,125.038000,125.158000,0.000000,0.000000,0.000000,-1.000000,0.000000,-4.474263,-2.963380,-1.000000e+02,124.833000,124.833000
25%,140.094000,140.209000,139.976000,140.092000,140.034000,140.150000,11.904762,57.142857,22.413793,-1.000000,0.037000,-0.044675,-0.047143,-6.029412e+01,140.094000,140.094000
50%,146.002000,146.128000,145.882000,146.003000,145.946000,146.067000,25.320513,74.679487,43.310520,1.000000,0.083000,0.005595,-0.002761,-6.674082e-01,146.002500,146.002000
75%,152.318000,152.428000,152.187000,152.320000,152.246000,152.382000,42.857143,88.095238,64.383562,1.000000,0.164000,0.051163,0.045662,1.526821e+02,152.314000,152.311000
max,172.050000,172.136000,171.664000,172.054000,171.712000,172.054000,100.000000,100.000000,100.000000,1.000000,5.415000,2.343088,2.676402,inf,171.875000,171.875000


In [11]:
HANGING_MAN_BODY = 15.0
HANGING_MAN_HEIGHT = 75.0
SHOOTING_STAR_HEIGHT = 25.0
SPINNING_TOP_MIN = 40.0
SPINNING_TOP_MAX = 60.0
MARUBOZU = 98.0
ENGULFING_FACTOR = 1.1

# Règle pour avoir un Marubozu (le corps occupe presque tout le chandelier) 
# Future changement de tendance si le corps occuple plus de 98% du chandelier
apply_marubozu = lambda x : x.body_perc > MARUBOZU

# Règles pour avoir un pendu ou un marteau
# Si pendu (rouge) future tendance haussière
# Si marteau (vert) future tendance baissière
def apply_hanging_man(row):
#   Le bas du corps doit être à plus de 75% du chandelier
#   Le corps ne doit pas avoir plus de 15% du chandelier
    if row.body_bottom_perc > HANGING_MAN_HEIGHT:
        if row.body_perc < HANGING_MAN_BODY:
            return True
        return False

# Règles pour avoir l'étoile filante ou le marteau inversé
# Si étoile filante (rouge) future tendance baissière
# Si marteau inversé (vert) future tendance haussière
def apply_shooting_star(row):
#   Le bas du corps doit être à moins de 25% du chandelier
#     Le corps ne doit pas avoir plus de 15% du chandelier
    if row.body_bottom_perc < SHOOTING_STAR_HEIGHT:
        if row.body_perc < HANGING_MAN_BODY:
            return True
        return False

# Règle pour le spinning top (le corps est très petit par rapport au chandelier)
# Si bearish spinning top (rouge) future tendance baissière
# Si bullish spinning top (vert) future tendance haussière
def apply_spinning_top(row):
#   Le corps doit être compris entre 40 et 60% du chandelier
#   Le corps ne doit pas avoir plus de 15% du chandelier
    if row.body_bottom_perc < SPINNING_TOP_MAX:
        if row.body_bottom_perc > SPINNING_TOP_MIN:
            if row.body_perc < HANGING_MAN_BODY:
                return True
    return False

# Règle pour les avalements (le corps du chandelier du jour couvre complètement celui du jour précédent)
# Bullish engulfing (Avalement haussier) : Rouge-vert implique une tendance haussière
# Bearish engulfing (Avalement baissier) : Vert-rouge implique une tendance baissière
def apply_engulfing(row):
    if row.direction != row.direction_prev:
        if row.body_size > row.body_size_prev * ENGULFING_FACTOR:
#       Pour le Harami
#       if row.body_size < row.body_size_prev * ENGULFING_FACTOR:
            return True
    return False
# La similitude d'un avalement est un Harami (le corps du jour est complètement recouvert par celui du jour précédent)
# Si on a rouge-vert le Harami est haussier
# Si on a vert-rouge le Harami est baissier


TWEEZER_BODY = 15.0
TWEEZER_HL = 0.01
TWEEZER_TOP_BODY = 40.0
TWEEZER_BOTTOM_BODY = 60.0

# Règle pour les tweezer (2 chandeliers successifs de corps presque identiques)
# Si les 2 chandeliers sont vert-rouge future tendance baissière
# Si les 2  chandeliers sont rouge-vert future tendance haussière
def apply_tweezer_top(row):
#   La variation du corps doit être inférieure à 15%
    if abs(row.body_size_change) < TWEEZER_BODY:
#       Le chandelier actuele doit être rouge et le précédent doit être de couleur différente
        if row.direction == -1 and row.direction != row.direction_prev:
#           Les variations entre les extremités doit être faible
            if abs(row.low_change) < TWEEZER_HL and abs(row.high_change) < TWEEZER_HL:
#                 L'extremité supérieure du corps doit être à moins de 40% du chandelier
                if row.body_top_perc < TWEEZER_TOP_BODY:
                    return True 
    return False 

def apply_tweezer_bottom(row):
#   La variation du corps doit être inférieure à 15%
    if abs(row.body_size_change) < TWEEZER_BODY:
#       Le chandelier actuele doit être vert et le précédent doit être de couleur différente
        if row.direction == 1 and row.direction != row.direction_prev:
#           Les variations entre les extremités doit être faible
            if abs(row.low_change) < TWEEZER_HL and abs(row.high_change) < TWEEZER_HL:
#                 L'extremité inférieure du corps doit être à plus de 60% du chandelier
                if row.body_bottom_perc > TWEEZER_BOTTOM_BODY:
                    return True 
    return False 

MORNING_STAR_PREV2_BODY = 90.0
MORNING_STAR_PREV_BODY = 10.0

def apply_morning_star(row, direction=1):
#   le corps du 3e dernier chandelier doit être supérieur à 90% du chandelier
    if row.body_perc_prev2 > MORNING_STAR_PREV2_BODY:
#       Le corps du 2e dernier chandelier doit etre inférieur à 10% du chandelier
        if row.body_perc_prev < MORNING_STAR_PREV_BODY:
#           Le dernier et le 3e dernier chandelier doivent être de couleur différente
            if row.direction == direction and row.direction_prev2 != direction:
#               Si le dernier chandelier est vert il faut que le haut de son corps (close) soit au dessus de la moitié du 3e dernier chandelier (mornnig)
                if direction == 1:
                    if row.mid_c > row.mid_point_prev_2:
                        return True
#               Si le dernier chandelier est rouge il faut que le bas de son corps (close) soit en dessous de la moitié du 3e dernier chandelier (evening)
                else:
                    if row.mid_c < row.mid_point_prev_2:
                        return True
    return False 

In [12]:
# On utilise shift pour créer une colonne à partir de l'avant-dernière valeur d'une colonne
df_an['body_size_prev'] =  df_an.body_size.shift(1)
df_an['direction_prev'] =  df_an.direction.shift(1)
df_an['direction_prev2'] =  df_an.direction.shift(2)
df_an['body_perc_prev'] =  df_an.body_perc.shift(1)
df_an['body_perc_prev2'] =  df_an.body_perc.shift(2)
# La méthode apply prend en entrée une fonction qui agit sur chaque ligne ou colonne
df_an['HANGING_MAN'] = df_an.apply(apply_hanging_man, axis=1)
df_an['SHOOTING_STAR'] = df_an.apply(apply_shooting_star, axis=1)
df_an['SPINNING_TOP'] = df_an.apply(apply_spinning_top, axis=1)
df_an['MARUBOZU'] = df_an.apply(apply_marubozu, axis=1)
df_an['ENFULGING'] = df_an.apply(apply_engulfing, axis=1)
df_an['TWEEZER_TOP'] = df_an.apply(apply_tweezer_top, axis=1)
df_an['TWEEZER_BOTTOM'] = df_an.apply(apply_tweezer_bottom, axis=1)
df_an['MORNING_STAR'] = df_an.apply(apply_morning_star, axis=1)
df_an['EVENING_STAR'] = df_an.apply(apply_morning_star, axis=1, direction=-1)

In [20]:
# Cette commande permet de retrouver les valeurs où le pattern est True afin de visualiser avec le graphe plus tard
df_an[df_an['EVENING_STAR'] == True]

,time,mid_o,mid_h,mid_l,mid_c,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,body_perc_prev2,HANGING_MAN,SHOOTING_STAR,SPINNING_TOP,MARUBOZU,ENFULGING,TWEEZER_TOP,TWEEZER_BOTTOM,MORNING_STAR,EVENING_STAR
2763,2017-06-13 01:00:00+00:00,139.270,139.436,139.128,139.202,139.202,139.270,24.025974,75.974026,22.077922,...,92.814371,None,False,False,False,False,False,False,False,True
7078,2018-02-21 19:00:00+00:00,150.360,150.502,150.124,150.171,150.171,150.360,12.433862,87.566138,50.000000,...,91.696751,None,False,False,False,True,False,False,False,True
9034,2018-06-15 06:00:00+00:00,146.788,146.819,146.635,146.654,146.654,146.788,10.326087,89.673913,72.826087,...,100.000000,None,False,False,False,True,False,False,False,True
20092,2020-03-26 21:00:00+00:00,133.752,133.778,133.044,133.091,133.091,133.752,6.403270,93.596730,90.054496,...,91.059908,None,False,False,False,False,False,False,False,True
21247,2020-06-03 01:00:00+00:00,136.900,137.010,136.462,136.504,136.504,136.900,7.664234,92.335766,72.262774,...,95.781250,None,False,False,False,False,False,False,False,True


In [17]:
import plotly.graph_objects as go

In [21]:
MARKER = '#0066FF'
dfp = df_an.iloc[20050:21250]
cp = CandlePlot(dfp, candles=True)

# Pour la lecture de chaque pattern il faut te rassurer que le chandelier en bleu correspond au dernier chandelier dans le pattern
df_temp = cp.df_plot[cp.df_plot.EVENING_STAR==True]
cp.fig.add_trace(go.Candlestick(
                x=df_temp.sTime,
                open=df_temp.mid_o,
                high=df_temp.mid_h,
                low=df_temp.mid_l,
                close=df_temp.mid_c,
                line=dict(width=1), opacity=1,
                increasing_fillcolor=MARKER,
                decreasing_fillcolor=MARKER,
                increasing_line_color=MARKER,
                decreasing_line_color=MARKER
            ))

cp.show_plot()